## Construire un pytree

### Qu'est-ce qu'un pytree ?


Un PyTree est une structure arborescente construite à partir d'objets Python enregistré dans le "registre des conteneurs". Dans ce registre il y a par défaut: `list`, `tuple`, `dict`, `namedtuple`.  

Mais on peut aussi créer des objets  et les ajouter dans ce registre.

Tout objet qui ne figure pas dans le registre est considéré comme une feuille du pytree.



Les pytree servent à enregistrer:

* Les paramètres d'un modèle: on construit un pytree dont les feuilles sont des tenseurs jax.
* Les paramètres d'un optimiseur
* Les data d'entrainement
* etc.


### Conteneurs `list`, `tuple`, `dict`

In [2]:
import jax
import jax.numpy as jnp

a_pytree=[
    1,
    {'k1': object(),'k2': (3, (4,5))}, #dictionnaire
    None,
    (), #un tuple vide,
    ("a",1),
    jnp.array([1, 2, 3])
]

Un pytree a une struture:

In [3]:
jax.tree.structure(a_pytree)

PyTreeDef([*, {'k1': *, 'k2': (*, (*, *))}, None, (), (*, *), *])

Et un contenu qui sont ses feuilles:

In [4]:
jax.tree.leaves(a_pytree)

[1, <object at 0x1107e9380>, 3, 4, 5, 'a', 1, Array([1, 2, 3], dtype=int32)]

Applatir un pytree, c'est donner les feuilles et la structure

In [5]:
jax.tree.flatten(a_pytree)

([1, <object at 0x1107e9380>, 3, 4, 5, 'a', 1, Array([1, 2, 3], dtype=int32)],
 PyTreeDef([*, {'k1': *, 'k2': (*, (*, *))}, None, (), (*, *), *]))

Désapplatir c'est crée un pytree à partir de ses feuilles et de sa structure:

In [6]:
vals, treedef = jax.tree.flatten([1, (2, 3), [4, 5]])

newvals = [100, 200, 300, 400, 500]

jax.tree.unflatten(treedef, newvals)

[100, (200, 300), [400, 500]]

⇑ on a créer un nouveau pytree en changeant les feuilles.

### conteneur `namedtuple`

Très pratique les namedtuple. On peut les construire avec une fonction:

In [7]:
from collections import namedtuple

Point_1 = namedtuple('Point', ['x', 'y'])

p = Point_1(x=1, y=2)

In [8]:
print(p[0],p[1])

1 2


Ils ont toutes les propriétés des tuples, mais en plus on peut appeler leurs éléments comme des attributs.

In [9]:
print(p.x, p.y)

1 2


On peut les construire par héritage:

In [10]:
from typing import NamedTuple

class Point_2(NamedTuple):
    x:int
    y:int

In [11]:
p = Point_2(x=1, y=2)
print(p.x, p.y)

1 2


In [12]:
print(p[0],p[1])

1 2


Vérifions que cela marche comme conteneur pour les pytree:

In [13]:
points=Point_2(x=jnp.zeros([3]),y=jnp.ones([3]))

jax.tree.structure(points)

PyTreeDef(CustomNode(namedtuple[Point_2], [*, *]))

In [14]:
jax.tree.leaves(points)

[Array([0., 0., 0.], dtype=float32), Array([1., 1., 1.], dtype=float32)]

### conteneur perso


Voilà comme déclarer notre propre classe comme conteneur.

In [16]:
from jax.tree_util import register_pytree_node

class Point_3:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __repr__(self):
        return f"Point(x={self.x}, y={self.y})"


#on explique comment applatir un objet de notre classe spéciale
def point_flatten(v):
    children = (v.x, v.y)
    aux_data = None # n'importe quoi qui pourrait nous aider.
    return (children, aux_data)


#On explique comment on construit notre objet à partir de sa version applatie.
def point_unflatten(aux_data, children):
    return Point_3(children[0],children[1])


# Global registration
register_pytree_node(
    Point_3,
    point_flatten,    # Instruct JAX what are the children nodes.
    point_unflatten   # Instruct JAX how to pack back into a `RegisteredSpecial`.
)


points=Point_3(x=jnp.zeros([3]),y=jnp.ones([3]))
jax.tree.structure(points)

PyTreeDef(CustomNode(Point_3[None], [*, *]))

In [17]:
jax.tree.leaves(points)

[Array([0., 0., 0.], dtype=float32), Array([1., 1., 1.], dtype=float32)]

##  Action sur les pytree

### Des feuilles avec des noms

In [18]:
import collections

points=Point_3(x=jnp.zeros([3]),y=jnp.ones([3]))
point =Point_1(x=1,y=3)

tree = [1, {'k1': 2, 'k2': (3, 4)}, points,point]

In [19]:
flattened, structure = jax.tree_util.tree_flatten_with_path(tree)

for key_path, value in flattened:
    print(key_path)
    print(f'Value of tree{jax.tree_util.keystr(key_path)}: {value}')
    print()

(SequenceKey(idx=0),)
Value of tree[0]: 1

(SequenceKey(idx=1), DictKey(key='k1'))
Value of tree[1]['k1']: 2

(SequenceKey(idx=1), DictKey(key='k2'), SequenceKey(idx=0))
Value of tree[1]['k2'][0]: 3

(SequenceKey(idx=1), DictKey(key='k2'), SequenceKey(idx=1))
Value of tree[1]['k2'][1]: 4

(SequenceKey(idx=2), FlattenedIndexKey(key=0))
Value of tree[2][<flat index 0>]: [0. 0. 0.]

(SequenceKey(idx=2), FlattenedIndexKey(key=1))
Value of tree[2][<flat index 1>]: [1. 1. 1.]

(SequenceKey(idx=3), GetAttrKey(name='x'))
Value of tree[3].x: 1

(SequenceKey(idx=3), GetAttrKey(name='y'))
Value of tree[3].y: 3



### map

In [20]:
a_pytree = [
    [1, 2, 3],
    {"a":1,"b": 2},
    [1, 2, 3, 4]
]

jax.tree.map(lambda x: x*2, a_pytree)

[[2, 4, 6], {'a': 2, 'b': 4}, [2, 4, 6, 8]]

Appliquer une fonction à 2 arguments sur 2 pytree:

In [21]:
another_pytree = a_pytree
jax.tree.map(lambda x, y: x+y, a_pytree, another_pytree)

[[2, 4, 6], {'a': 2, 'b': 4}, [2, 4, 6, 8]]

### Dériver

In [22]:
def fn(point):
    return point.x**2 * point.y**3

point = Point_3(3.,1.)
point

Point(x=3.0, y=1.0)

In [23]:
jax.grad(fn)(point)

Point(x=6.0, y=27.0)

***A vous:*** Introduisez un bug ou un print dans la fonction `point_unflatten` pour vérifiez que `grad` va bien l'utiliser.

## pytree de tenseur

Les paramètres de nos modèles seront des pytree de tenseurs.

Mais aussi souvent, les inputs de nos modèle, quand il faut les structurer, sont des pytree de tenseurs.

In [24]:
import jax.random as jr

In [25]:
layer_widths=(2,5,9,3)
rkey=jr.key(0)
params = []
for n_in, n_out in zip(layer_widths[:-1], layer_widths[1:]):
    rk,rkey=jr.split(rkey)
    params.append(
        {"weight":jr.normal(rk,shape=(n_in, n_out))*jnp.sqrt(2/n_in),
        "bias":jnp.zeros([n_out])})

In [26]:
for leaves in jax.tree.leaves(params):
    print(leaves.shape)

(5,)
(2, 5)
(9,)
(5, 9)
(3,)
(9, 3)


In [27]:
from jax.flatten_util import ravel_pytree

params_flat, unflatten_fn = ravel_pytree(params)

In [28]:
params_flat.shape

(99,)

In [29]:
params_back=unflatten_fn(params_flat)
for leaves in jax.tree.leaves(params_back):
    print(leaves.shape)

(5,)
(2, 5)
(9,)
(5, 9)
(3,)
(9, 3)


## Défi prog

Il s'agit de créer une fonction `select_pytree_from_subpath` qui permette d'extraire un sous-pytree. Par exemple considérons:

        tree_input = (1, {"a": (2,[5,6]), "c": [3, 4]})


On crée un sélector: un pytree de booléen dont la structure est incluse dans celle du pytree initial


        tree_selector = (True, {"a": False, "c": True})


Le résultat sera le `tree_input` auquel on aura coupé toutes les branches qui correspondent à un `False` dans le `tree_selector`


        tree_output = select_pytree_from_subpath
        (tree_input, tree_selector,False)


Le `tree_output` sera:

        (1, {'a': None, 'c': [3, 4]})


Aide: Voici le début de ma solution. Mais si vous voulez partir sur une autre idée ...

In [30]:
def select_pytree_from_subpath(pytree,tree_selector,replace_leaves_by_TrueFalse):

    flattened, structure = jax.tree_util.tree_flatten_with_path(pytree)
    flattened_selector, _ = jax.tree_util.tree_flatten_with_path(tree_selector)
    keys_selector=[kv[0] for kv in flattened_selector if kv[1]]

    def is_super_key(k):
        for key_selector in keys_selector:
            if k[:len(key_selector)]==key_selector:
                return True
        return False

    flattened_selected=[]
    ...

    tree_selected=jax.tree.unflatten(structure,flattened_selected)
    return tree_selected

Voici le programme test:

In [31]:
def test():
    tree_input = (1, {"a": 2, "c": [3, 4]})

    tree_selector = (False, {"a": True, "c": True})
    tree_output=select_pytree_from_subpath(tree_input,tree_selector,False)
    assert str(tree_output)=="(None, {'a': 2, 'c': [3, 4]})"
    tree_selector = (False, {"a": True, "c": True})
    tree_output = select_pytree_from_subpath(tree_input, tree_selector, True)
    assert str(tree_output) == "(False, {'a': True, 'c': [True, True]})"


    tree_selector = (True, {"a": False, "c": True})
    tree_output = select_pytree_from_subpath(tree_input, tree_selector,False)
    assert str(tree_output)=="(1, {'a': None, 'c': [3, 4]})"
    tree_output = select_pytree_from_subpath(tree_input, tree_selector, True)
    assert str(tree_output) == "(True, {'a': False, 'c': [True, True]})"

    tree_selector = (True, {"c": True})
    tree_output = select_pytree_from_subpath(tree_input, tree_selector,False)
    assert str(tree_output) == "(1, {'a': None, 'c': [3, 4]})"

    tree_selector = (True, {"a": True, "c": False})
    tree_output = select_pytree_from_subpath(tree_input, tree_selector,False)
    assert str(tree_output) == "(1, {'a': 2, 'c': [None, None]})"
    tree_output = select_pytree_from_subpath(tree_input, tree_selector,True)
    assert str(tree_output) == "(True, {'a': True, 'c': [False, False]})"

test()

ValueError: Too few leaves for PyTreeDef; expected 4, got 0